## KNN (K-Nearest-Neighbors)

KNN is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item, by having them somehow "vote" on it.

As an example, let's look at the MovieLens data. We'll try to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, we'll load up every rating in the data set into a Pandas DataFrame:

In [12]:
# bbilootecas usadas
import pandas as pd
import numpy as np

# Gera uma tabela com as informações de filmes contidas no arquivo
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('/home/ronyus/Documentos/card - 10/Aquivos de Código/More Data Mining and Machine Learning Techniques/Dados/u.data',sep='\t', names=['user_id', 'movie_id', 'rating'], usecols=range(3))
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie:

In [4]:
# Agrupamento dos filmes por id e contagem de quantos filmes foram avaliados e a média
movieProperties = ratings.groupby('movie_id').agg({'rating': ['size', 'mean']})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [5]:
# Normalização dos dados
# Cria uma tabela com a quantidade de avaliações por filme
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
# Aplica uma função lambda para normalizar os dados, utilizando o número máximo e mínimo de avaliações
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()
# Um filme com 0.0 significa que ele foi avaliado o mínimo de vezes, e 1.0 o máximo de vezes

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [10]:
# Agrupamento dos filmes por id e contagem de quantos filmes foram avaliados e a média
movieProperties = ratings.groupby('movie_id').agg({'rating': ['size', 'mean']})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [6]:
# Cria uma tabela com a quantidade de avaliações por filme
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
# Aplica uma função lambda para normalizar os dados
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:

In [8]:
movieDict = {}
with open(r'/home/ronyus/Documentos/card - 10/Aquivos de Código/More Data Mining and Machine Learning Techniques/Dados/u.item', encoding='ISO-8859-1') as f:
    temp = ''
    for line in f:
        # O código percorrerá o arquivo linha por linha resgatando as informaçõe
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25] 
        # Os gêneros são representados por um vetor de 19 posições onde cada número representa um gênero diferente
        genres = map(int, genres)
        movieDict[movieID] = (
            name,
            np.array(list(genres)),
            movieNormalizedNumRatings.loc[movieID].get('size'),
            movieProperties.loc[movieID].rating.get('mean')
        )

For example, here's the record we end up with for movie ID 1, "Toy Story":

In [9]:
# onde cada posição do array representa um gênero diferente
print(movieDict[1])

('Toy Story (1995)', array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), np.float64(0.7735849056603774), np.float64(3.8783185840707963))


Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:

In [10]:
from scipy import spatial

def ComputeDistance(a, b):
    # resgata os gêneros dos filmes a partir de cada filme
    genresA = a[1]
    genresB = b[1]
    # Calcula a distância entre os gêneros dos filmes utilizando a distância cosseno
    genreDistance = spatial.distance.cosine(genresA, genresB)
    # após isso será feita a comparação da popularidade dos filmes
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA - popularityB)
    return genreDistance + popularityDistance
    # Se o valor é próximo a 0 significa que os filmes são similares, e quanto maior o valor, menos similares são
    
ComputeDistance(movieDict[2], movieDict[4])

np.float64(0.8004574042309892)

Remember the higher the distance, the less similar the movies are. Let's check what movies 2 and 4 actually are - and confirm they're not really all that similar:

In [11]:
print(movieDict[2])
print(movieDict[4])

('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), np.float64(0.22298456260720412), np.float64(3.2061068702290076))
('Get Shorty (1995)', array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), np.float64(0.3567753001715266), np.float64(3.550239234449761))


Now, we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our data set. When the sort those by distance, and print out the K nearest neighbors:

In [13]:
import operator

# Função que retorna os K vizinhos mais próximos de um filme
def getNeighbors(movieID, K):
    distances = []
    #dalcula a distância entre o filme Toy Story e todos os outros filmes
    for movie in movieDict:
        if (movie != movieID):
            # Utiliza a função criada anteriormente
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            # Adiciona a distância e o id do filme a um array
            distances.append((movie, dist))
    # Ordena o array de distâncias de forma crescente
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    # Adiciona os K vizinhos mais próximos ao array de vizinhos
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors
# Retorna os 10 vizinhos mais próximos do filme Toy Story
K = 10
avgRating = 0
neighbors = getNeighbors(1, K)
# Para cada vizinho, imprime o nome do filme e a média de avaliações
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3])) 
avgRating /= K

Liar Liar (1997) 3.156701030927835
Aladdin (1992) 3.8127853881278537
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Monty Python and the Holy Grail (1974) 4.0664556962025316
Full Monty, The (1997) 3.926984126984127
George of the Jungle (1997) 2.685185185185185
Beavis and Butt-head Do America (1996) 2.7884615384615383
Birdcage, The (1996) 3.4436860068259385
Home Alone (1990) 3.0875912408759123
Aladdin and the King of Thieves (1996) 2.8461538461538463


While we were at it, we computed the average rating of the 10 nearest neighbors to Toy Story:

In [14]:
avgRating

np.float64(3.3445905900235564)

How does this compare to Toy Story's actual average rating?

In [15]:
movieDict[1][3]

np.float64(3.8783185840707963)

Not too bad!